In [2]:
import numpy
from etrade.spot.trader import Station
from etrade.spot.market.recycle import BasicRecycle, PointwiseRecycle
from etrade.spot.forecast.market import DistributiveSeries, DistributiveMarket

from data_utils.stochastic_utils.vdistributions.parameter.continuous.basic import NormalDistribution, LogNormalDistribution, eps
from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.histogram import HistogramDistribution
from data_utils.stochastic_utils.vdistributions.nonparametric.discrete.basic import DiscreteDistribution
from data_utils.stochastic_utils.vdistributions.tools.convert import resample_like_distribution, generate_correlated_sample_matrix

from matplotlib import pyplot as plt

from scipy.optimize import differential_evolution

from matplotlib import pyplot

In [6]:
s = Station("station", 50)
monte_carlo_num = 1000


In [13]:
def real_market_data(m=(25, 10), s=(5,1), data_len = 4, num = 100, sample_num = 10):
    series = []
    mean = NormalDistribution(*m).rvf(data_len * num)
    std = NormalDistribution(*s).rvf(data_len * num)
    for i in range(sample_num):
        series.append(NormalDistribution(mean[i], std[i]))
    return DistributiveSeries(*series)

In [15]:
real_market_data((1,1), (0.1,0.5)).rvf(10)

array([[ 0.95110035, -0.70009192,  2.14013272,  2.32677152, -0.40871419,
         2.12216103,  1.03534041,  2.39166189,  1.52715539,  2.52610013],
       [-0.28188333, -0.03082209, -0.16861967, -0.1031414 , -0.22927118,
        -0.0737026 , -0.1600746 , -0.34163384, -0.10911408, -0.25309962],
       [ 2.42383368,  2.11544058,  2.26357483,  2.12599415,  2.22085992,
         2.51187449,  2.10108411,  2.32076206,  2.22714559,  2.04046588],
       [ 1.11301797,  0.79395175,  1.14254513,  1.49479392,  1.38342179,
         1.99592652,  0.707089  ,  1.6869823 ,  1.7741101 ,  0.89475431],
       [ 0.35000382,  0.49710357,  0.75753728,  0.1849027 ,  0.76439982,
        -0.06311   ,  0.78501974,  0.28411999, -0.02832578,  1.11594388],
       [ 1.14626798,  0.95904527,  0.82657744,  1.14833009,  0.92727175,
         0.95384279,  1.53548886,  1.12905852,  1.33972309,  0.88361384],
       [ 1.45749172,  1.27330912, -0.60387313, -0.17669528, -0.76781574,
         0.64654649,  0.78884461,  0.90648473